In [13]:
import os
import torch
from torch import nn
from torch.utils.data import dataloader
from torchvision import datasets,transforms

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else 'cpu'
print('Device : ',device)

Device :  cpu


In [14]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )
        
    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [15]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [16]:
x = torch.rand(1,28,28,device=device)
logits = model(x)
pred_prob = nn.Softmax(dim=1)(logits)
y_pred = pred_prob.argmax(1)
print(f"Predicted class : {y_pred}")

Predicted class : tensor([1])


In [17]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [18]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [19]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [20]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 5.9980e-03,  4.0272e-01, -4.4242e-02,  3.5729e-01,  3.7366e-02,
         -6.4091e-01,  1.5439e-01, -3.3680e-01,  3.6516e-01, -1.0618e-01,
          7.7226e-02,  9.1221e-02,  1.6771e-01,  1.4266e-02,  9.4313e-02,
          2.6848e-01, -2.8255e-01,  1.5646e-02, -3.5235e-01,  2.6231e-01],
        [ 1.3664e-01,  3.9197e-01,  1.4279e-01,  4.8672e-01,  2.7704e-01,
         -5.5741e-01,  2.3743e-01, -6.9766e-01,  2.1167e-01, -5.3622e-01,
         -1.4136e-01,  1.2038e-01,  1.1259e-01,  9.9965e-02,  1.9670e-01,
         -1.2108e-01, -2.1853e-01, -1.3227e-01, -3.8003e-01, -2.8552e-01],
        [ 1.5561e-01,  6.2905e-01, -3.1800e-01,  3.1856e-01, -2.4620e-01,
         -5.2880e-01, -2.6480e-01, -3.4062e-01,  1.2572e-02, -2.7911e-04,
          1.1380e-01,  3.6205e-02, -3.6788e-02,  2.1687e-01, -1.3758e-01,
          3.7302e-01, -4.1136e-01, -6.6935e-02, -1.9304e-01,  3.9487e-04]],
       grad_fn=<AddmmBackward0>)


After ReLU: tensor([[5.9980e-03, 4.0272e-01, 0.0000e+00, 3.5

In [21]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [22]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [23]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0329, -0.0044, -0.0094,  ...,  0.0258,  0.0283,  0.0009],
        [-0.0227,  0.0332, -0.0168,  ..., -0.0033, -0.0012,  0.0041]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0113,  0.0254], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0251, -0.0397,  0.0437,  ..., -0.0102,  0.0415,  0.0230],
        [ 0.0348, -0.0315,  0.0106,  ..., -0.0249, -0.0348,  0.0187]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | 